In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn import preprocessing
import matplotlib.pyplot as plt
from datetime import datetime

import numpy as np
import math

data = pd.read_csv('LABS-LAGGED.csv')
data['Timestamp'] = pd.to_datetime(data['Timestamp'], format='%Y-%m-%d')
data.set_index('Timestamp', inplace=True)

data.drop(columns=['DayOfWeek','HSW-COD-load','HSW-COD-load_1', 'HSW-COD-load_2', 'HSW-COD-load_3', 'HSW-COD-load_4',
                  'HSW-COD-load_5', 'HSW-COD-load_6', 'Dig-stability', 'Biogas_6', 'TWAS-VS-load_2', 'PS-VS-load_3'
                  , 'PS-VS-load_5', 'PS-VS-load_6', 'TWAS-VS-load_4','TWAS-VS-load_3','SRT','PS-VS-load_1','HSW-VS-load_2'
                  , 'Biogas_4','HSW-VS-load_5','PS-VS-load','Biogas_3','PS-VS-load_2','TWAS-VS-load_5','TWAS-VS-load_6'
                  , 'PS-VS-load_4', 'Biogas_2','HSW-VS-load_3','HSW-VS-load_4','TWAS-VS-load','TWAS-VS-load_1','BOD-load'], inplace=True)

data.columns

Index(['Biogas', 'HSW-VS-load', 'Forecast', 'HSW-VS-load_1', 'HSW-VS-load_6',
       'Biogas_1', 'Biogas_5'],
      dtype='object')

In [2]:
from sklearn.preprocessing import StandardScaler

x_train, x_test, y_train, y_test = train_test_split(
    data.drop('Forecast', axis=1), data['Forecast'], test_size=0.2,
    shuffle=False)

# Initialize scaler object
scaler = StandardScaler()

# Fit the scaler on the training dataset
scaler.fit(x_train)

# Transform the training and testing dataset
x_train_s = pd.DataFrame(scaler.transform(x_train), index=x_train.index)
x_test_s = pd.DataFrame(scaler.transform(x_test), index=x_test.index)

# Add the column names back
x_train_s.columns = x_train.columns
x_test_s.columns = x_test.columns

# Check to make sure everything looks right
x_train_s

,Biogas,HSW-VS-load,HSW-VS-load_1,HSW-VS-load_6,Biogas_1,Biogas_5
Timestamp,,,,,,
2020-01-07,-1.546131,-1.165609,-1.161616,-1.139907,-1.041926,-0.900309
2020-01-08,-1.219688,-1.165609,-1.161616,-1.139907,-1.540460,-1.137155
2020-01-09,-1.280591,-1.165609,-1.161616,-1.139907,-1.214589,-1.405420
2020-01-10,-1.085700,-1.165609,-1.161616,-1.139907,-1.275386,-1.632599
2020-01-11,-1.073519,-1.165609,-1.161616,-1.139907,-1.080836,-1.021149
...,...,...,...,...,...,...
2022-08-20,0.103137,0.416764,0.423844,-0.398416,0.538789,-0.105182
2022-08-21,-0.722715,0.014252,0.418684,0.615089,0.105916,0.428931
2022-08-22,-0.384091,1.109677,0.016700,1.290171,-0.718488,0.513519


In [3]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import TimeSeriesSplit

cv=TimeSeriesSplit(gap = 5)
model_lm = RidgeCV(cv=cv) 

model_lm.fit(x_train_s, y_train)

# Predict using the fitted model
y_train_lm = model_lm.predict(x_train_s)
y_test_lm = model_lm.predict(x_test_s)

# Calculate training and testing error
mse_train_lm = mean_squared_error(y_train, y_train_lm)
mape_train_lm = mean_absolute_percentage_error(y_train, y_train_lm)
r2_train_lm = r2_score(y_train, y_train_lm)
rmse_train = math.sqrt(mse_train_lm)
mse_test_lm = mean_squared_error(y_test, y_test_lm)
mape_test_lm = mean_absolute_percentage_error(y_test, y_test_lm)
r2_test_lm = r2_score(y_test, y_test_lm)
rmse_test = math.sqrt(mse_test_lm)

# Print training and testing error
print(f"Training Mean Squared Error (MSE): {round(mse_train_lm,3)}")
print(f"Training Root Mean Squared Error (MSE): {round(rmse_train,3)}")
print(f"Training Mean Absolute Percentage Error (MAPE): {round(mape_train_lm,3)}")
print(f"Training R-squared (R2) Score: {round(r2_train_lm,3)}")
print(f"Testing Mean Squared Error (MSE): {round(mse_test_lm,3)}")
print(f"Testing Root Mean Squared Error (MSE): {round(rmse_test,3)}")
print(f"Testing Mean Absolute Percentage Error (MAPE): {round(mape_test_lm,4)}")
print(f"Testing R-squared (R2) Score: {round(r2_test_lm,3)}")

def adjusted_r2(r2, n, k):
    return 1 - ((1 - r2) * (n - 1)) / (n - k - 1)

n = len(x_test_s)  # Number of samples in test data
k = len(x_train_s.columns)  # Number of predictors in the model

# Calculate Adjusted R-squared
adjusted_r2_value = adjusted_r2(r2_test_lm, n, k)
print("Adjusted R-squared:", round(adjusted_r2_value,4))

TeEI = (mape_test_lm*mape_test_lm*rmse_test)/adjusted_r2_value
print("TeEI:", round(TeEI,3))

Training Mean Squared Error (MSE): 501.77
Training Root Mean Squared Error (MSE): 22.4
Training Mean Absolute Percentage Error (MAPE): 0.348
Training R-squared (R2) Score: 0.701
Testing Mean Squared Error (MSE): 589.715
Testing Root Mean Squared Error (MSE): 24.284
Testing Mean Absolute Percentage Error (MAPE): 0.2762
Testing R-squared (R2) Score: 0.623
Adjusted R-squared: 0.6125
TeEI: 3.025


In [4]:
#Option to add the linear prediction as a new feature in the error model

#y_train_lm2 = pd.DataFrame(y_train_lm, columns=["Linear_pred"], index=y_train.index)
#y_test_lm2 = pd.DataFrame(y_test_lm, columns=["Linear_pred"], index=y_test.index)
#x_train_s2 = pd.concat([x_train_s, y_train_lm2], axis=1)
#x_test_s2 = pd.concat([x_test_s,y_test_lm2],axis=1)
#x_train_s2.shape

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor

y_train_err = y_train - y_train_lm
y_test_err = y_test - y_test_lm

# Set the random seed
random_seed = 1
# Initialize Model
model_ann2 = MLPRegressor(max_iter=500,
                          early_stopping=True,
                          random_state=random_seed,
                          #hidden_layer_sizes=160,
                          #solver='sgd',
                          #learning_rate='adaptive',
                          #alpha=0.01
                         )

# Create a 'grid' of ANN model fitting parameters to explore
param_grid = {
    'hidden_layer_sizes': [ (2,), (6,), (10,), (15,), (4,2), (6,3), (8,4), (10,5), (10,2), (20,2), (65,2), (20,5), (65,5), (20,10), (65,10)],
    'alpha': [0.01,0.1, 1,5,10,20,30,40, 50,100,500],
    'learning_rate': ['invscaling', 'adaptive'],
    'solver': ['sgd', 'adam']
}

grid_search = GridSearchCV(model_ann2,
                           param_grid,
                           cv=cv,
                           scoring='neg_mean_absolute_percentage_error'
                           )

# Conduct a 'grid search' for the 'best' model
grid_search.fit(x_train_s.values, y_train_err)

# Print the best results
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", -grid_search.best_score_)

best_model = grid_search.best_estimator_
y_train_ann2 = best_model.predict(x_train_s.values)
y_test_ann2 = best_model.predict(x_test_s.values)

y_train_hybrid = y_train_lm + y_train_ann2
y_test_hybrid = y_test_lm + y_test_ann2

# Calculate training and testing error
mse_train_hyb = mean_squared_error(y_train, y_train_hybrid)
mape_train_hyb = mean_absolute_percentage_error(y_train, y_train_hybrid)
rmse_train = math.sqrt(mse_train_hyb)
r2_train_hyb = r2_score(y_train, y_train_hybrid)
mse_test_hyb = mean_squared_error(y_test, y_test_hybrid)
mape_test_hyb = mean_absolute_percentage_error(y_test, y_test_hybrid)
r2_test_hyb = r2_score(y_test, y_test_hybrid)
rmse_test = math.sqrt(mse_test_hyb)

# Print training and testing error
print(f"Training Mean Squared Error (MSE): {round(mse_train_hyb,3)}")
print(f"Training Root Mean Squared Error (MSE): {round(rmse_train,3)}")
print(f"Training Mean Absolute Percentage Error (MAPE): {round(mape_train_hyb,3)}")
print(f"Training R-squared (R2) Score: {round(r2_train_hyb,3)}")
print(f"Testing Mean Squared Error (MSE): {round(mse_test_hyb,3)}")
print(f"Testing Root Mean Squared Error (MSE): {round(rmse_test,3)}")
print(f"Testing Mean Absolute Percentage Error (MAPE): {round(mape_test_hyb,4)}")
print(f"Testing R-squared (R2) Score: {round(r2_test_hyb,3)}")

def adjusted_r2(r2, n, k):
    return 1 - ((1 - r2) * (n - 1)) / (n - k - 1)

n = len(x_test_s)  # Number of samples in test data
k = len(x_train_s.columns)  # Number of predictors in the model

# Calculate Adjusted R-squared
adjusted_r2_value = adjusted_r2(r2_test_hyb, n, k)
print("Adjusted R-squared:", round(adjusted_r2_value,4))
print("Number of variables:",k)
TeEI = (mape_test_hyb*mape_test_hyb*rmse_test)/adjusted_r2_value
print("TeEI:", round(TeEI,3))

/usr/local/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptro

Best parameters:  {'alpha': 500, 'hidden_layer_sizes': (10, 2), 'learning_rate': 'invscaling', 'solver': 'adam'}
Best score:  1.0166991567729144
Training Mean Squared Error (MSE): 501.77
Training Root Mean Squared Error (MSE): 22.4
Training Mean Absolute Percentage Error (MAPE): 0.348
Training R-squared (R2) Score: 0.701
Testing Mean Squared Error (MSE): 589.724
Testing Root Mean Squared Error (MSE): 24.284
Testing Mean Absolute Percentage Error (MAPE): 0.276
Testing R-squared (R2) Score: 0.623
Adjusted R-squared: 0.6125
Number of variables: 6
TeEI: 3.02
